In [1]:
import scipy.linalg as la
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import numpy as np
from helper_functions.grape_functions import *
from main_grape.Grape import Grape
%pylab inline



Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:



#Defining time scales
total_time = 5.0
steps = 500

state_transfer = True
#Defining H0
qubit_state_num = 2
qubit_num = 2
freq_ge = 3.9#GHz
g = 0.2 #GHz

ens = np.array([ 2*np.pi*ii*(freq_ge) for ii in np.arange(qubit_state_num)])
Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
Q_z   = np.diag(np.arange(0,qubit_state_num))
I_q = np.identity(qubit_state_num)
H_q = np.diag(ens)

X = multi_kron(Q_x,qubit_num)
Z = multi_kron(Q_z,qubit_num)
Y = multi_kron(Q_y,qubit_num)

H0 = kron_all(H_q,qubit_num, I_q)# + g*X


#Defining Concerned states (starting states)



all_ones_index = 0
for ii in range (qubit_num):
    all_ones_index = all_ones_index + qubit_state_num**ii
#Defining states to include in the drawing of occupation
states_draw_list = [0,all_ones_index]
states_draw_names = ['00..0','11..1']

psi0=[0,all_ones_index]

#Defining U (Target)

target_vec_0 = np.zeros(qubit_state_num**qubit_num,dtype=complex)
target_vec_0[all_ones_index]=1

target_vec_1 = np.zeros(qubit_state_num**qubit_num,dtype=complex)
target_vec_1[0]=1
    
U = [target_vec_0, target_vec_1]

#Defining U0 (Initial)
q_identity = np.identity(qubit_state_num**qubit_num)
U0= q_identity

#Defining control Hs

Hops = []
Hnames = []
ops_max_amp = []
Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=8.0)
Hops,Hnames,ops_max_amp = append_separate_krons(Q_z,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=8.0)
Hops.extend([X,Z])
Hnames.extend(['all x','all z'])
ops_max_amp.extend([8.0,8.0])

print Hnames

#Defining convergence parameters
max_iterations = 1000
decay = max_iterations/2
convergence = {'rate':0.001, 'update_step':1, 'max_iterations':max_iterations,\
               'conv_target':1e-4,'learning_rate_decay':decay}

states_forbidden_list = [all_ones_index]

reg_coeffs = {'dwdt':0.0001,'d2wdt2':0.0001*0.001, 'forbidden':10000,
             'states_forbidden_list': states_forbidden_list}


['xi', 'ix', 'zi', 'iz', 'all x', 'all z']


In [ ]:
 #Extra options
#reg_coeffs = reg_coeffs,draw = [states_draw_list,states_draw_names] ,forbidden = states_forbidden_list,multi_mode = multi_mode,
#maxA = ops_max_amp,use_gpu = True

uks,U_final = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names],
                    state_transfer=True,
                    show_plots = True, 
       unitary_error = 1e-6, method = 'L-BFGS-B',switch = False,  maxA=ops_max_amp,\
                   reg_coeffs=reg_coeffs, file_name='spin_chain_ghz',
                    data_path = '/home/nelson/Simulations/GRAPE-Data/spin_chain_ghz')
      
      